In [5]:
import torch
from __future__ import division

In [26]:
def bbox_iou(box1, box2, x1y1x2y2=True):
    
    if not x1y1x2y2:
        b1_x1, b1_x2 = box1[:, 0] - box1[:, 2] / 2, box1[:, 0] + box1[:, 2] / 2
        b1_y1, b1_y2 = box1[:, 1] - box1[:, 3] / 2, box1[:, 1] + box1[:, 3] / 2
        b2_x1, b2_x2 = box2[:, 0] - box2[:, 2] / 2, box2[:, 0] + box2[:, 2] / 2
        b2_y1, b2_y2 = box2[:, 1] - box2[:, 3] / 2, box2[:, 1] + box2[:, 3] / 2
    else:
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[:,0], box1[:,1], box1[:,2], box1[:,3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[:,0], box2[:,1], box2[:,2], box2[:,3]

    inter_rect_x1 =  torch.max(b1_x1, b2_x1)
    inter_rect_y1 =  torch.max(b1_y1, b2_y1)
    inter_rect_x2 =  torch.min(b1_x2, b2_x2)
    inter_rect_y2 =  torch.min(b1_y2, b2_y2)

    inter_area =    torch.clamp(inter_rect_x2 - inter_rect_x1 , min=0) * \
                    torch.clamp(inter_rect_y2 - inter_rect_y1 , min=0)
        
    b1_area = (b1_x2 - b1_x1 ) * (b1_y2 - b1_y1 )
    b2_area = (b2_x2 - b2_x1 ) * (b2_y2 - b2_y1 )

    iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)

    return iou


In [34]:
#x1y1x2y2
a = torch.tensor([[0.,0.,2.,2.]])
b = torch.tensor([[1.,1.,3.,3.],[0.,0.,2.,2.]])
#c1c2wh
c = torch.tensor([[1.,1.,2.,2.]])
d = torch.tensor([[2.,2.,2.,2.]])

In [22]:
1/12

0.08333333333333333

In [35]:
bbox_iou(a,b,True)

tensor([ 0.1429,  1.0000])

In [28]:
import numpy as np
a = np.array([1,2,3])
b = np.array([0,a])

In [30]:
a

array([1, 2, 3])

In [38]:
def build_targets(pred_boxes, target, anchors, num_anchors, num_classes, dim, ignore_thres, img_dim):
    #这个函数的作用是把标记框编码到某一尺寸的fmap上去
    #target是标注信息
    nB = target.size(0)  #batch个数  16
    nA = num_anchors     #锚框个数   3
    nC = num_classes     #数据集类别数  80
    dim = dim            #feature map的尺寸，即13*13的13

    mask        = torch.zeros(nB, nA, dim, dim)     #[16,3,13,13]   全0
    conf_mask   = torch.ones(nB, nA, dim, dim)      #[16,3,13,13]   全1
    tx          = torch.zeros(nB, nA, dim, dim)     #[16,3,13,13]   全0
    ty          = torch.zeros(nB, nA, dim, dim)     #[16,3,13,13]   全0
    tw          = torch.zeros(nB, nA, dim, dim)     #[16,3,13,13]   全0
    th          = torch.zeros(nB, nA, dim, dim)     #[16,3,13,13]   全0
    tconf       = torch.zeros(nB, nA, dim, dim)     #[16,3,13,13]   全0
    tcls        = torch.zeros(nB, nA, dim, dim, num_classes)    #[16,3,13,13,80]  全0
    nGT = 0  # 统计 真值框个数 GT ground truth
    nCorrect = 0  
    # 统计 预测出有物体的个数 （即 真值框 与 3个原始锚框与真值框iou最大的那个锚框对应的预测框  之间的iou > 0.5 为预测正确）

    for b in range(nB):#16张图遍历
        for t in range(target.shape[1]):#每张图的bbox遍历
            nGT += 1
            gx = target[b, t, 1] * dim
            gy = target[b, t, 2] * dim
            gw = target[b, t, 3] * dim
            gh = target[b, t, 4] * dim
            gi = int(gx)
            gj = int(gy)
            gt_box = torch.FloatTensor(np.array([0, 0, gw, gh])).unsqueeze(0)
            anchor_shapes = torch.FloatTensor(np.concatenate((np.zeros((len(anchors), 2)), np.array(anchors)), 1))
            anch_ious = bbox_iou(gt_box, anchor_shapes)
            conf_mask[b, anch_ious > ignore_thres] = 0
            best_n = np.argmax(anch_ious)
            
            gt_box = torch.FloatTensor(np.array([gx, gy, gw, gh])).unsqueeze(0)
            pred_box = pred_boxes[b, best_n, gj, gi].unsqueeze(0)
            mask[b, best_n, gj, gi] = 1
            conf_mask[b, best_n, gj, gi] = 1
            tx[b, best_n, gj, gi] = gx - gi
            ty[b, best_n, gj, gi] = gy - gj
            tw[b, best_n, gj, gi] = math.log(gw/anchors[best_n][0] + 1e-16)
            th[b, best_n, gj, gi] = math.log(gh/anchors[best_n][1] + 1e-16)
            tcls[b, best_n, gj, gi, int(target[b, t, 0])] = 1
            iou = bbox_iou(gt_box, pred_box, x1y1x2y2=False)
            # [16,3,13,13]   全0，有真值框对应的网格位置为1  标明 物体中心点落在该网格中，该网格去负责预测物体
            tconf[b, best_n, gj, gi] = 1

            if iou > 0.5:
                nCorrect += 1
    return nGT, nCorrect, mask, conf_mask, tx, ty, tw, th, tconf, tcls